# Выпуклые оболочки

Множество называется **выпуклым**, если, для любых двух точек, все точки, лежащие между ними, тоже принадлежат этому множеству.

**Выпуклой оболочкой** множества точек называется такое выпуклое множество, которое включает их всех, и при этом является минимальным по площади.

Зачем они нужны? На самом деле, с выпуклыми множествами очень легко работать. Например, нас могут спрашивать про какие-нибудь свойства наборов точек, например, искать 

Очень часто выпуклые оболочки возникают в задачах, казалось бы, совсем не связанных с геометрией.

Сейчас мы научимся её быстро строить, то есть получать из множества точек их выпуклую оболочку, как-то упорядоченную по обходу. Для упрощения будем считать, что никакие три точки не лежат на одной прямой.

## Построение за $O(n^2)$

Посмотрим на самую левую точку (а если таких много — то на самую нижнюю из них). Она точно будет в выпуклой оболочке. Теперь посмотрим на все остальные точки и мысленно отсортируем их по полярному углу относительно уже включенной. Последняя из сортированных точно будет лежать в оболочке, причем сразу после текущей. Возьмем её тоже, и будем идти так дальше, пока оболочка не построится.

Такой алгоритм будет для каждой точки в итоговой выпуклой оболочке просматривать по $O(n)$ точек, что в худшем случае будет суммарно $O(n^2)$.

Это называют алгоритмом Джарвиса. Кодить не будем, сразу перейдем к оптимизированной версии.

## Построение за $O(n \log n)$

Оптимизация этого алгоритма основывается на использовании информации об изначальной сортировке.

На каждую вершину мы посмотрим максимум два раза — когда будем её добавлять, и, возможно, когда будем удалять. Получается, тут в асимптотике логарифм только из-за изначальной сортировки.

Это называют алгоритмом Грэхэма.

https://www.youtube.com/watch?v=BTgjXwhoMuI

Небольшой ликбез: в стандартную сортировку можно передать *компаратор* — специальную функцию, которая используется вместо стандартного `<`. Она должна принимать два объекта нужного типа и возвращать `true`, если первый «меньше» второго.

Ещё в C++ и вообще во многих языках есть концепция лямбда-функций: они объявляются.

In [0]:
vector<r> convex_hull (vector<int> points) {
    r z = points[0];
    for (r p : points)
        if (p.x < z.x || (p.x == z.x && p.y < z.y))
            z = p;
    sort(points.begin(), points.end(), [&](r a, r b){
        return (a-z) ^ (b-z) > 0;
    })
    vector<r> hull;
    for (r p : points) {
        while (hull.size() >= 2 && hull[hull.size()-1] - ) {
            hull.pop_back();
        }
    }
}

Иногда требуется поддерживать операции добавления. Тогда имеет смысл разделить оболочку на две части — верхнюю и нижнюю огибающую. Вообще, их кодить проще.

Есть ещё всякие более сложные алгоритмы. Есть алгоритм Чена, который работает O(n \log h) и как бы объединяет лучшее из двух миров. Есть ещё алгоритм, позволяющий добавлять и удалять точки за $O(n \log^2 n)$ и занимает более 400 строк, и там есть один жуткий бинпоиск, в котором рассматривается 7 случаев, и вообще является одним из самых мерзких алгоритмов в мире. Ну да ладно, перейдем к задачам.

## Фриланс

> У фрилансера Серёжи есть две мечты: стать опытным специалистом (получить $A$ опыта) и купить на квартиру в Москве (накопить $B$ рублей). У него есть $n \leq 10^5$ предложений о работе, $i$-е из которых приносит в час $a_i$ опыта и $b_i$ рублей. Работать над каждой задачей он может произвольное количество времени, не обязательно выражающееся целым количеством часов. Какое наименьшее время он может потратить, чтобы осуществить обе свои мечты?

Имеет ли смысл брать работу?

Все рассуждения, которые мы сделали — это идея выпуклой оболочки, если перевести в геометрию.

Получается, всегда достаточно брать какую-то комбинацию из двух работ.

На самом деле, часто нас просят на самом деле найти не выпуклую оболочку, а верхнюю или нижнюю огибающую, что уже проще. 

## Convex Hull Trick

CHT — довольно мощная оптимизация динамики. Обычно она заключается в том, чтобы внимательно посмотреть на рекурренту и внезапно увидеть там минимизацию какого-нибудь скалярного произведения, что сводится к нахождению касательной в выпуклой оболочке.

Рассмотрим конкретную задачу:

> Даны  n  точек на прямой. Нужно найти  m  отрезков, покрывающих все точки, минимизировав при этом сумму квадратов их длин.

Как бы более-менее понятно, как должно выглядеть базовое решение — это должна быть следующая динамика: $f[i, j]$ — минимальная стоимость покрытия $i$ первых (самых левых) точек, используя не более $j$ отрезков (итоговый ответ будет записан в $f[n, m]$ — прим. К. О.).

Переход — перебор всех возможных последних отрезков, то есть $f[i, j] = \min_{k < i} \{f[k, j-1] + (x_{i-1}-x_k)^2 \}$.

In [0]:
// x[] — отсортированный массив координат точек, нумерация с нуля

// квадрат длины отрезка от i-той до j-той точки
int cost(int i, int j) { return (x[j]-x[i])*(x[j]-x[i]); }

for (int i = 0; i <= m; i++)
    f[0][k] = 0; // если нам не нужно ничего покрывать, то всё и так хорошо
// все остальные f предполагаем равными бесконечности

for (int i = 1; i <= n; i++)
    for (int j = 1; j <= m; j++)
        for (int k = 0; k < i; k++)
            f[i][j] = min(f[i][j], f[k][j-1] + cost(k, i-1));

Теперь распишем выражение для $f[i, j]$:

$$
f[i, j] = \min_{k < i} \{f[k, j-1] + (x_{i-1}-x_k)^2 \} = \min_{k < i} \{
f[k, j-1] + x_{i-1}^2
- 2x_{i-1} x_k
+ x_k^2
\}
$$

Посмотрим внимательнее на минимизируемое выражение. $x_{i-1}^2$ не зависит от $k$, значит его можно вынести. Под минимумом останется
$
\underbrace{f[k, j-1] + x_k^2}_{a_k}
\underbrace{-2x_k}_{b_k} x_{i-1}
$.

Это теперь можно переписать как $
\min_k
(a_k, b_k)
\cdot
(1, X_{i-1})
$ (имеется в виду скалярное произведение)

Представим $(a_k, b_k)$ как точки на плоскости. Тогда мы можем построить их нижнюю огибающие и бинарным поиском находить оптимальную, с минимальным скалярным произведением.

In [0]:
// TODO: написать код

<картинка со Скуби-Ду>